In [1]:
import mercury as mr
from reportlab.pdfgen import canvas

## Document structure
Let's look at a couple of documents to see if there is a consistent structure we can try to rely on...

In [42]:
mr.PDF("docs/112517929Blackfin_USB_LAN_EZ_Extender_Manual_Rev._2.0.pdf", width="100%", height=400)

In [43]:
mr.PDF("docs/UG-277.pdf", width="100%", height=400)

We see the structure of files is different. The first is a manual. The second is a user guide. Let's see how many manuals exist.

In [36]:
import pandas as pd
import subprocess

result = subprocess.run(['ls docs | grep -iE "manual*"'], shell=True, capture_output=True, text = True)
file_list = result.stdout.splitlines()
count = 1
for f in file_list:
    print (str(count) + ": " + f)
    count = count +1

1: 112517929Blackfin_USB_LAN_EZ_Extender_Manual_Rev._2.0.pdf
2: 20819009ADSP_BF535_EZ_KIT_LiteManual.pdf
3: 3379713228940ADSP_BF537EZ_KTKITLiteEvaluationSystemManualRev_1_1.pdf
4: 34499303127081ADSP_BF537_EZ_KIT_Lite_Manual_Rev20.pdf
5: 359818064ADSP_BF561_EZ_KIT_Lite_Manual_Rev.1_2.pdf
6: 38886150643836Blackfin_A_V_EZ_Extender_Manual_Rev_1.0.pdf
7: 39262143042901Blackfin_EZ_Extender_Manual_Rev_3.0.pdf
8: 3943519977326Blkf_FPGA_extender_manual.pdf
9: 39582370881972Blkf_Audio_extender_manual.pdf
10: 39724380433683Blackfin_USB_LAN_EZ_Extender_Manual_Rev_1.0.pdf
11: 397386663422ADSP_BF561_EZ_KIT_Lite_Manual_Rev._3.1.pdf
12: 402257434ADSP_BF533_EZ_KIT_Lite_Manual_Rev._1_3.pdf
13: 40980416611920ADSP_BF561_EZ_KIT_Lite_Manual_Rev_2.0.pdf
14: 51653280154286ADSP_BF535_EZ_KIT_Lite_Manual_Rev_3.0.pdf
15: 577304032113756464ADSP_BF535_EZ_KIT_Lite_Manual_Rev._3.1.pdf
16: 59552369272925ADSP_BF561_EZ_KIT_Lite_Manual_Rev_30.pdf
17: 6164764637251ADSP_BF537_EZ_KIT_Lite_Manual_Rev._2.2.pdf
18: 62687914348

Let's try another manual...

In [46]:
mr.PDF("docs/ICE_emu_1000_2000_rev_manual.pdf", width="100%", height=400)